# Scraping OECD's Glossary of Statistical Terms and writing the results to the Content Database

### Revised (January 2022) to correct an error

### https://stats.oecd.org/glossary/

In [1]:
import urllib.request as urllib2 
from urllib.request import urlopen

import bs4
from bs4 import BeautifulSoup

import re
import pandas as pd

import requests

import numpy as np
from operator import itemgetter

### Scraping

* Because there are blanks in the IDs of the Glossary articles, we scan all IDs up to a number large enough.
* The sections are not standard. The various names met are used to delimit the text.

In [2]:
base_url = "https://stats.oecd.org/glossary/detail.asp?ID={}"

sections = ['Definition','Definition:','Statistical Theme:','Context:',
            'Last updated on','Created on','Cross References:','Source Publication:','French Equivalent:','Hyperlink:','Glossary Output Segments:','Version Indicator:','Classification Indicator:','French Definition:']


num_articles = 8000
OECD_df=pd.DataFrame(index=range(num_articles))

for n in range(1,num_articles+1):
    if n % 100 ==0: print(n)
    scrape_url = base_url.format(n)
    OECD_df.loc[n,'ID'] = n
    OECD_df.loc[n,'URL'] = scrape_url
    res = requests.get(scrape_url)
    if res.status_code != 200: continue
    soup = bs4.BeautifulSoup(res.content,'lxml')
    
    #links = []
    #for link in soup.findAll('a'):
    #    links.append(link.get('href'))

    #str1 = " ".join(str(x) for x in links)
    #p=re.findall(r'detail\.asp\?ID=\d+', str1)
    #p = ['https://stats.oecd.org/glossary/'+lnk for lnk in p]
    
    links = soup.findAll('a',href=re.compile(r'detail\.asp\?ID=\d+'))
    p = ['https://stats.oecd.org/glossary/'+l.get('href') for l in links]
    
    text=soup.get_text()
    text=re.sub(r'(\n)+',' ',text)
    text=re.sub(r'( )+',' ',text)
    text=text.replace('\xa0','')
    text=text.replace('\r',' ')
    text=re.sub(r'( )+',' ',text)
    text = text.replace('OECD Glossary of Statistical Terms -', "")
    text = text.replace('Glossary Home About Contact Us Downloadable Version Advanced Filter Web Service OECD Statistics', "")



    first_column = True ## first to be collected: Term
    matches = [(re.search(section,text).span()[0],re.search(section,text).span()[1],section) 
               for section in sections if not re.search(section,text) is None]
    matches.sort(key=itemgetter(0))
    for k in range(len(matches)):
        match=matches[k] 
        s1,s2 = match[:2] ## start-end
        column = match[2] ## item from list 'sections'

        if k == len(matches)-1: ## last one, end of text to collect is end of 'text'
            s1_next = len(text)
        else:                   ## not last one, end of text to collect is start of next match  
            match_next=matches[k+1]
            s1_next = match_next[0]
        if first_column:
            cand_term = text[:s1].strip()
            test = re.search('"> ',cand_term) ## an anomaly in https://stats.oecd.org/glossary/detail.asp?ID=2216
            if test is not None:
                _,sb = test.span(0)
                cand_term = cand_term[(sb+1):].strip()
            OECD_df.loc[n,'Term'] = cand_term
            print(n,OECD_df.loc[n,'Term'])
            first_column = False ## finished with Term - will be reset to True in next 'text'
        else:    
            OECD_df.loc[n,column] = text[s2:s1_next].strip()
        #print(p)
        OECD_df.loc[n,'URL:Cross References']=','.join(p)
        
OECD_df_keep =  OECD_df.copy(deep=True) ## just a copy to be able to change things without re-running this time-consuming part

1 Abatement
2 Absence from work due to illness
3 Activity restriction - free expectancy
4 Acute care
5 Acute care beds
6 Administrative data collection
7 Administrative source
8 Abnormal obsolescence
10 Academic staff
11 Academic support for students
12 Accessibility (as a statistical data quality dimension)
13 Account
14 Accountability (environmental)
16 Accrual accounting
18 Accrued interest
19 Accumulation
20 Accumulation accounts
21 Accuracy
22 Acid rain
23 Aquaculture
24 Acquisitions
25 Acquisition price
26 Acquisition, time of
28 Active labour market programmes
29 Active metainformation system
30 Activity
32 Activity classification
33 Actual final consumption of general government
34 Actual final consumption of households
35 Actual final consumption of non-profit institutions serving households
36 Actual individual consumption
37 Actual social contributions
40 Acute health conditions
41 Adaptation
42 Adapted products
43 ADB
44 Add-factor
45 Additivity - PPI
46 Adjusted disposable

351 Classification of environment protection activities (CEPA)
352 Classification of individual consumption by purpose (COICOP)
353 Classification of outlays of producers by purpose (COPP)
354 Classification of the functions of government (COFOG)
355 Classification of the purposes of non-profit institutions (COPNI)
356 Classification unit
357 Classification of visitors
358 Classification scheme
359 Climate
360 Climate change
362 Climate Convention
363 Climate protection
364 Closed ecological system
365 Closed-end investment company
366 Closed season
367 Cluster samples
368 Coal, oil and natural gas reserves
369 Coarse grains
371 Codex Alimentarius
372 Codex Alimentarius Commission
373 Coefficient table
374 COFOG (classification of the functions of government)
375 Coherence
376 Cohort
377 COICOP (classification of individual consumption by purpose)
378 Collective consumption service
379 Collective households
380 Collective living quarters
382 Co-management
383 Command-and-control policy

682 Due-for-payment recording
683 Dump
684 Dumping (of waste)
685 Dumping at sea
686 Durability of a building
687 Durable good
688 Duration of a fixed income instrument
689 Duration of marriage
690 Duration of stay
691 Duration of unemployment
692 Dust
693 Dust arrester
694 Dwelling - SNA
695 Dwelling – UN
696 Dwelling construction (activity)
697 Dwelling occupancy status
698 Dynamic Asian Countries
699 Dystrophic water
700
700 EAGGF
701 Early foetal death
702 Early retirement for labour market reasons
704 Earnings (wages and salaries) – ILO
705 Earnings (wages and salaries) – UN
706 Earnings per share
707 Earth Summit
708 Earthwatch
709 ECB
710 Eco-development
711 Eco-domestic product
712 Ecological amplitude
713 Ecological balance
714 Ecological dominance
715 Ecological equilibrium
716 Ecological ethics
717 Ecological footprint
718 Ecological impact
719 Ecological statistics
720 Ecology
721 Economic assets
723 Economic instruments (environmental protection policy)
724 Economic intere

1000 Flag of convenience countries
1001 Flag state (for fishing vessel)
1002 Flexible inputs
1004 Floor area
1005 Flow series / data
1006 Economic flows
1007 Flows in real terms
1008 Fluorocarbon
1009 Fob price
1011 Foetal death rate
1012 Follow-up survey (or multi-round survey or multi-phase survey)
1013 Food and Agriculture Organisation (FAO)
1014 Food balance sheet
1015 Food chain
1016 Food web
1017 Foot and mouth disease
1018 Foreign affiliate
1019 Foreign assets
1020 Foreign bank
1021 Foreign border workers
1022 Foreign-born population of a country
1023 Foreign business travellers
1024 Foreign controlled corporations (non- financial and financial)
1026 Foreign currency transactions (banking)
1027 Foreign diplomatic and consular personnel
1028 Foreign direct investment
1031 Foreign direct investment enterprise
1032 Foreign exchange companies
1033 Foreign exchange rate
1034 Foreign exchange reserves
1035 Foreign exchange swap
1036 Financial guarantee corporations
1037 Foreigners adm

1293 Implicit price index based on constant exchange rates (or constant international prices) of period t0
1294 Implicit price index based on constant exchange rates (or constant international prices) of period t
1295 Import coverage ratio
1296 Import duties
1297 Import quotas
1298 Import subsidies
1299 Imports of goods and services – SNA
1300
1300 Imports of goods and services – UN
1302 Import price index
1303 Improvised housing units
1305 Imputed expenditure
1306 Imputed social contributions
1307 Inbound tourism
1308 Inbound tourism consumption
1309 Incineration
1310 Incineration at sea
1311 Incineration with recovery of energy
1312 Incinerator
1313 Income – SNA
1314 Income – Eurostat
1315 Net income from abroad
1317 Income on debt (interest accrued)
1318 Income on equity
1319 Incomes of health and social workers (average)
1320 Incomes of physicians, general practitioners, dentists
1321 Incorporated enterprise
1322 Incremental product innovation
1324 Laspeyres index number
1325 Index

...

6031 Upper-middle-income countries
6032 Write-off
6033 Cover
6034 Enhanced structural adjustment facility (ESAF)
6035 Structural adjustment facility (SAF)
6036 Poverty reduction and growth facility (PRGF)
6037 Houston terms
6038 Extended fund facility (EFF)
6039 Flow rescheduling
6040 Debt default
6041 London Club
6042 Nominal value (of a debt instrument)
6043 Official development assistance (ODA)
6044 Concessionality level - IMF
6045 Net present value (NPV) of debt
6046 Short-term debt
6047 Long-term external debt
6048 Final maturity date
6049 American depository receipt (ADR)
6050 Balances on nostro and vostro accounts
6051 Bank deposits
6052 Bearer depository receipts (BDR)
6053 Collateralised debt obligations (CDOs)
6054 Commercial paper
6055 Commodity linked bonds
6056 Commodity linked derivatives
6057 Convertible bonds
6058 Credit linked note
6059 Currency linked bonds
6060 Currency pool loans
6061 Depository receipts
6062 Deposits in mutual associations
6063 Equity linked bond
6

6345 Accounting treatment of terminal costs
6346 Acidifying potential (AP)
6347 Additions to stock levels
6348 Aggregation in physical accounts
6349 Agricultural land and surface water (SEEA)
6350 Allocation of emissions to final demand
6351 Amenity functions of the natural capital
6352 Annual fellings
6353 Annual removals
6354 Appropriation method
6355 Avoidance
6356 Benefit transfer
6357 Bequest benefit
6358 Best available technology
6359 Biological resources
6360 Capital approach to sustainable development
6361 Capital expenditure for environmental protection
6362 Capital services method
6363 Capture fisheries
6364 CEPA 2000
6365 Changes in land cover by categories of changes
6366 Changes in the rate of extraction
6367 Changes in the unit resource rent
6368 Cleaner technologies and products group (of the environment industry)
6369 Competition of functions
6370 Compliance criterion
6371 Conjoint analysis approaches
6372 Connected products
6374 Consumption value method (for standing t

6580 Stumpage price
6581 Stumpage value method (for standing timber valuation)
6582 Supply and use table for environmental protection
6583 Supply and use table for natural resources
6584 Supply and use table for residuals
6585 Sustainable catch
6586 Sustainable development indicator
6587 Sustainable national income
6588 Sustainable yield
6589 Symmetric treatment of defensive expenditure
6590 Terminal costs
6591 Three pillar approach to sustainable development
6592 Total (domestic) Material input
6593 Total actual renewable water resources
6594 Total Domestic Output (TDO)
6595 Total material consumption (TMC)
6596 Total Material Output (TMO)
6597 Total material requirement (TMR)
6598 Total natural renewable water
6599 Total non–renewable water
6600
6600 Travel cost method
6601 Unused extraction
6602 Use benefits
6603 Use tables for natural resources
6604 Valuation of the degradation of land and soil
6605 Valuing natural resources
6606 Virtual population analysis
6607 Volume
6608 Water q

6869 New products (in context of innovation)
6870 Process innovation
6871 Marketing innovation
6872 Innovations in pricing
6873 Organisational innovation
6874 Innovative active firm
6875 Acquisition of technology and knowledge
6876 Open information sources
6877 Innovation co-operation
6878 Knowledge management
6879 Ambiguity rule
6880 Analysis server
6881 Attribution
6882 Anonymised data
6883 Anonymised record
6884 Approximate disclosure
6885 Argus
6886 Attribute disclosure
6887 Barnardisation
6888 Blurring
6889 Bounds
6890 Calculated interval
6891 Cell suppression
6892 Complementary suppression
6893 Complete disclosure
6894 Concentration rule
6895 Confidentiality edit
6896 Controlled rounding
6897 Controlled Tabular Adjustment (CTA)
6898 Conventional rounding
6899 Data divergence
6900
6900 Data intruder
6901 Data intrusion detection
6902 Data Intrusion Simulation (DIS)
6903 Data protection
6904 Data swapping
6905 Data utility
6906 Deterministic rounding
6907 Direct identification
6908

7210 EMAS
7211 Environmental R&D
7212 Environment-commercial 'win-wins'
7213 Flexible policy measures
7214 Market-based instruments
7215 Performance-based standard
7216 Pollution prevention
7217 Prescriptive policy measures
7218 Technology-based standards
7219 Total suspended solids (TSS)
7220 Biobank
7221 Human genetic research database(s) (HGRDs)
7222 Biological sample
7223 Coded samples
7224 Consent
7225 Identified samples
7226 Unidentified samples
7227 Unlinked samples
7228 Benchmark
7229 Capacity assessment
7230 Capacity development
7231 Civil society organisations
7232 Country assistance strategies / plans
7233 Development Policy Lending
7234 Direct Budget Support (DBS)
7235 Fragile States
7236 Governance
7237 Good governance
7238 National ownership
7239 Policy reform
7240 Sector wide approach
7241 Sectoral strategy
7242 Strategic conflict assessment
7243 Structural Adjustment Programmes
7244 Tiering
7245 Arm’s length principle
7246 Due care
7247 Heightened managerial care
7248 H

In [18]:
OECD_df = OECD_df_keep.copy(deep=True) 

OECD_df.dropna(subset = ['Term','Definition:'],inplace=True)
OECD_df.reset_index(drop=True,inplace=True)
OECD_df['ID'] = OECD_df['ID'].astype('int32')

OECD_df.to_excel('OECD_df.xlsx')
OECD_df.head()

,ID,URL,Term,URL:Cross References,French Equivalent:,Definition:,Cross References:,Statistical Theme:,Created on,Last updated on,Source Publication:,Context:,Hyperlink:,Glossary Output Segments:,Classification Indicator:,Version Indicator:,French Definition:
0,1,https://stats.oecd.org/glossary/detail.asp?ID=1,Abatement,https://stats.oecd.org/glossary/detail.asp?ID=...,Réduction,See Pollution abatement.,Pollution abatement,Environmental statistics,"Tuesday, September 25, 2001","Thursday, March 14, 2002",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,https://stats.oecd.org/glossary/detail.asp?ID=2,Absence from work due to illness,,NaN,Absence from work due to illness refers to the...,NaN,Health statistics,"Tuesday, September 25, 2001","Thursday, November 22, 2001",OECD Health Data 2001: A Comparative Analysis ...,NaN,NaN,NaN,NaN,NaN,NaN
2,3,https://stats.oecd.org/glossary/detail.asp?ID=3,Activity restriction - free expectancy,,NaN,Functional limitation-free life expectancy is ...,NaN,Health statistics,"Tuesday, September 25, 2001","Wednesday, October 31, 2001",OECD Health Data 2001: A Comparative Analysis ...,NaN,NaN,NaN,NaN,NaN,NaN
3,4,https://stats.oecd.org/glossary/detail.asp?ID=4,Acute care,https://stats.oecd.org/glossary/detail.asp?ID=...,NaN,Acute care is one in which the principal inten...,Acute care beds Acute care hospital staff rati...,Health statistics,"Tuesday, September 25, 2001","Thursday, April 25, 2013",OECD Health Data 2001: A Comparative Analysis ...,NaN,NaN,NaN,NaN,NaN,NaN
4,5,https://stats.oecd.org/glossary/detail.asp?ID=5,Acute care beds,https://stats.oecd.org/glossary/detail.asp?ID=...,NaN,Acute care beds are beds accommodating patient...,Acute care Long-term care beds in hospitals,Health statistics,"Tuesday, September 25, 2001","Thursday, April 25, 2013",2001 Data Collection on Education Systems: Def...,Acute care beds have alternatively been define...,NaN,NaN,NaN,NaN,NaN


### Check cross-references

* Reason: there are cases where the cross-references lead to non-existing IDs.
* Add column 'Cross_References_2' with cross-references separated by semicolons.

In [19]:
for i in range(len(OECD_df)):

    if not OECD_df.loc[i,'URL:Cross References'].strip()=='':
        links = OECD_df.loc[i,'URL:Cross References'].split(',')

        matches=[re.search(r'\d+$',el) for el in links]

        x = [int(el[m.span()[0]:m.span()[1]]) for (el,m) in zip(links,matches)]
        ## x = [id for id in x if id in OECD_df['ID'].values.tolist()] ## if scraping partial results
        titles = [OECD_df.loc[np.where(OECD_df.ID==id)[0],'Term'].values[0] for id in x]
        OECD_df.loc[i,'Cross_References_2']=';'.join([t for t in titles if not t is np.nan])
        
OECD_df.head()        

,ID,URL,Term,URL:Cross References,French Equivalent:,Definition:,Cross References:,Statistical Theme:,Created on,Last updated on,Source Publication:,Context:,Hyperlink:,Glossary Output Segments:,Classification Indicator:,Version Indicator:,French Definition:,Cross_References_2
0,1,https://stats.oecd.org/glossary/detail.asp?ID=1,Abatement,https://stats.oecd.org/glossary/detail.asp?ID=...,Réduction,See Pollution abatement.,Pollution abatement,Environmental statistics,"Tuesday, September 25, 2001","Thursday, March 14, 2002",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pollution abatement
1,2,https://stats.oecd.org/glossary/detail.asp?ID=2,Absence from work due to illness,,NaN,Absence from work due to illness refers to the...,NaN,Health statistics,"Tuesday, September 25, 2001","Thursday, November 22, 2001",OECD Health Data 2001: A Comparative Analysis ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,https://stats.oecd.org/glossary/detail.asp?ID=3,Activity restriction - free expectancy,,NaN,Functional limitation-free life expectancy is ...,NaN,Health statistics,"Tuesday, September 25, 2001","Wednesday, October 31, 2001",OECD Health Data 2001: A Comparative Analysis ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,https://stats.oecd.org/glossary/detail.asp?ID=4,Acute care,https://stats.oecd.org/glossary/detail.asp?ID=...,NaN,Acute care is one in which the principal inten...,Acute care beds Acute care hospital staff rati...,Health statistics,"Tuesday, September 25, 2001","Thursday, April 25, 2013",OECD Health Data 2001: A Comparative Analysis ...,NaN,NaN,NaN,NaN,NaN,NaN,Acute care beds;Acute care hospital staff rati...
4,5,https://stats.oecd.org/glossary/detail.asp?ID=5,Acute care beds,https://stats.oecd.org/glossary/detail.asp?ID=...,NaN,Acute care beds are beds accommodating patient...,Acute care Long-term care beds in hospitals,Health statistics,"Tuesday, September 25, 2001","Thursday, April 25, 2013",2001 Data Collection on Education Systems: Def...,Acute care beds have alternatively been define...,NaN,NaN,NaN,NaN,NaN,Acute care;Long-term care beds in hospitals


### Some cleaning of the data

In [20]:
import unicodedata

OECD_df.drop(columns=['French Equivalent:', ## put also 'French Definition:'
                     'Glossary Output Segments:','Classification Indicator:','Version Indicator:',
                     'Created on','Hyperlink:'],inplace=True)
OECD_df.rename(columns={'Term':'term','Definition:':'definition',
                        'Statistical Theme:':'theme','Cross_References_2':'related','Context:':'context',
                        'URL:Cross References':'related_URL','Last updated on':'last_update'},inplace=True)

OECD_df.fillna(value='',inplace=True)
print(OECD_df.isnull().sum())


OECD_df['term']= OECD_df['term'].apply(lambda x: unicodedata.normalize('NFKD',x)) ## for the dashes in terms
#OECD_df['related']= OECD_df['related'].apply(lambda x: unicodedata.normalize('NFKD',x)) ## for the dashes in terms


OECD_df.head()

ID                    0
URL                   0
term                  0
related_URL           0
definition            0
Cross References:     0
theme                 0
last_update           0
context               0
French Definition:    0
related               0
dtype: int64


,ID,URL,term,related_URL,definition,Cross References:,theme,last_update,context,French Definition:,related
0,1,https://stats.oecd.org/glossary/detail.asp?ID=1,Abatement,https://stats.oecd.org/glossary/detail.asp?ID=...,See Pollution abatement.,Pollution abatement,Environmental statistics,"Thursday, March 14, 2002",,,Pollution abatement
1,2,https://stats.oecd.org/glossary/detail.asp?ID=2,Absence from work due to illness,,Absence from work due to illness refers to the...,,Health statistics,"Thursday, November 22, 2001",,,
2,3,https://stats.oecd.org/glossary/detail.asp?ID=3,Activity restriction - free expectancy,,Functional limitation-free life expectancy is ...,,Health statistics,"Wednesday, October 31, 2001",,,
3,4,https://stats.oecd.org/glossary/detail.asp?ID=4,Acute care,https://stats.oecd.org/glossary/detail.asp?ID=...,Acute care is one in which the principal inten...,Acute care beds Acute care hospital staff rati...,Health statistics,"Thursday, April 25, 2013",,,Acute care beds;Acute care hospital staff rati...
4,5,https://stats.oecd.org/glossary/detail.asp?ID=5,Acute care beds,https://stats.oecd.org/glossary/detail.asp?ID=...,Acute care beds are beds accommodating patient...,Acute care Long-term care beds in hospitals,Health statistics,"Thursday, April 25, 2013",Acute care beds have alternatively been define...,,Acute care;Long-term care beds in hospitals


### Local file for inspection

In [21]:
import datetime
current_time = datetime.datetime.now() 
outfile = 'OECD_final_results_2_'+str(current_time.month)+ '_' + str(current_time.day) + '_' + str(current_time.hour)+ '_' + str(current_time.minute)  +'.xlsx'

OECD_df.to_excel(outfile)


### Write to the database

In [16]:
%%script false --no-raise-error

import pyodbc

## the definition of the table
##create table "ESTAT"."V1"."OECD_Glossary" 
##( 
##  "id" INTEGER, 
##  "article_id" INTEGER, <- this is OECD's id 
##  "term" VARCHAR, 
##  "url" VARCHAR, 
##  "definition" LONG VARCHAR, 
##  "context" LONG VARCHAR, 
##  "theme" VARCHAR, 
##  "related" VARCHAR, 
##  "related_url" VARCHAR, 
##  "last_update" VARCHAR, 
##  "source_publ" VARCHAR,
##  PRIMARY KEY ("id") 
##); 

c = pyodbc.connect('DSN=Virtuoso All;DBA=ESTAT;UID=xxxxx;PWD=xxxxx')

sql = """INSERT INTO ESTAT.V1.OECD_Glossary (id,article_id,term,url,definition,context,theme,related,related_url,last_update,source_publ)
         VALUES (?,?,?,?,?,?,?,?,?,?,?)"""


cursor = c.cursor()

for i in range(len(OECD_df)):
    cursor.execute(sql,i+1,
        int(OECD_df.loc[i,'ID']),
        OECD_df.loc[i,'term'],    
        OECD_df.loc[i,'URL'],    
        OECD_df.loc[i,'definition'],    
        OECD_df.loc[i,'context'],    
        OECD_df.loc[i,'theme'],    
        OECD_df.loc[i,'related'],    
        OECD_df.loc[i,'related_URL'], 
        OECD_df.loc[i,'last_update'], 
        OECD_df.loc[i,'Source Publication:'])
    

c.commit()